(ch_shapes)=
# 形状

在 {ref}`ch_vector_add_te` 中定义的向量加法模块只接受长度为 100 的向量。对于输入可以具有任意形状的实际场景来说，这种限制太过严格。在本节中，我们将展示如何放松这个约束来处理一般情况。

## 形状变量

记住，我们为张量 `A` 和 `B` 创建了符号（symbolic）占位符，这样我们以后就可以 feed 数据。我们也可以对这个形状做同样的事情。特别地，下面的代码块使用 `te.var` 来为 `int32` 标量创建符号变量（symbolic variable），其值可以稍后指定。

In [1]:
from tvm_book.contrib import d2ltvm

import numpy as np
import tvm
from tvm import te

In [2]:
n = te.var(name='n')
type(n), n.dtype

(tvm.tir.expr.Var, 'int32')

可以使用 `(n,)` 为任意长度的向量创建占位符。

In [3]:
A = te.placeholder((n,), name='a')
B = te.placeholder((n,), name='b')
C = te.compute(A.shape, lambda i: A[i] + B[i], name='c')
s = te.create_schedule(C.op)
m = tvm.lower(s, [A, B, C], simple_mode=True)
m["main"]

PrimFunc([a, b, c]) attrs={"from_legacy_te_schedule": (bool)1, "global_symbol": "main", "tir.noalias": (bool)1} {
  for (i, 0, n) {
    c[(i*stride)] = (a[(i*stride)] + b[(i*stride)])
  }
}

与 {ref}`ch_vector_add_te` 中生成的伪代码相比，可以看到 for 循环的上限值从 100 变为了 `n`。

现在，我们像之前一样定义类似的测试函数，以验证编译后的模块能够正确地在不同长度的输入向量上执行。

In [4]:
def test_mod(mod, n):
    a, b, c = d2ltvm.get_abc(n, tvm.nd.array)
    mod(a, b, c)
    print('c.shape:', c.shape)
    np.testing.assert_equal(c.numpy(), a.numpy() + b.numpy())

mod = tvm.build(m)
test_mod(mod, 5)
test_mod(mod, 1000)

c.shape: (5,)
c.shape: (1000,)


但请注意，我们仍然设置了约束条件，即 `A`、`B` 和 `C` 必须处于相同的形状。因此，如果不满足，就会出现错误。

## 多维的形状

您可能已经注意到形状是以元组的形式呈现的。单个元素元组意味着一维张量，或者向量。我们可以通过在形状元组中添加变量将其扩展到多维张量。

下面的方法构建了用于多维张量加法的模块，维数由 `ndim` 指定。对于二维张量，可以通过 `A[i,j]` 来访问它的元素，类似地，对于三维张量，可以通过 `A[i,j,k]` 来访问它的元素。注意，在下面的代码中，使用 `*i` 来处理一般的多维情况。

In [5]:
def tvm_vector_add(ndim):
    A = te.placeholder([te.var() for _ in range(ndim)])
    B = te.placeholder(A.shape)
    C = te.compute(A.shape, lambda *i: A[i] + B[i])
    s = te.create_schedule(C.op)
    m = tvm.lower(s, [A, B, C])
    return tvm.build(m)

验证它是否适用于向量以外的情况。

In [6]:
mod = tvm_vector_add(2)
test_mod(mod, (2, 2))

mod = tvm_vector_add(4)
test_mod(mod, (2, 3, 4, 5))

c.shape: (2, 2)
c.shape: (2, 3, 4, 5)


## 小结

- 当在执行前不知道具体的数据形状时，可以使用 `te.var()` 来指定形状的维数。
- $n$ 维张量的形状被表示为 $n$ 长度的元组。